
main 함수 정의

In [133]:
#Hyper Parameters

RND_MEAN = 0        #정규분포 난숫값의 평균
RND_STD = 0.0030    #정규분포 난숫값의 표준편차

LEARNING_RATE = 0.001 

def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x,weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose() 

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis =0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

def pulsar_exec(epoch_count=10, mb_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)   #데이터셋 Load
    init_model()            #모델 파라미터 초기화
    train_and_test(epoch_count, mb_size, report)        #학습 및 평가 과정 수행 

In [134]:
def load_pulsar_dataset(adjust_ratio):
    
    pulsars, stars = [], []
    with open('data/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)   #next함수는 csv.reader 다음에 자주쓰이는 함수임 다음 행을 읽는다 뭐 이런 의미라고 하는데 궁금하면 Googling 해보기
        rows = []

        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1         #입출력 벡터 크기
    
    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')
    

후처리 과정에 대한 순전파와 역전파 함수(backprop) 재정의

In [135]:
import numpy as np 

def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux 

    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy

    return G_output 

시그모이드 관련 함수 정의

In [136]:
def relu(x):
    return np.maximum(x,0)

def sigmoid(x):
    return np.exp(-relu(-x))/ (1.0 + np.exp(-np.abs(x)))

def sigmoid_derv(x, y):
    return y * (1-y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x*z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z +sigmoid(x)
    

In [137]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [138]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data() 

    for epoch in range(epoch_count):
        losses =  []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, _ = run_train(train_x, train_y)
            losses.append(loss)

        if report > 0 and (epoch+1) % report ==0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {}: loss={:5.3f}, result={}'.format(epoch+1, np.mean(losses), acc_str))

    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    #final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final RESULT = {}'.format(acc_str))

학습 및 평가 데이터 획득 함수 정의

In [139]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count 

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]


def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]




학습 실행 함수와 평가 실행 함수 정의

In [140]:
def run_train(x,y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)

    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy 

def run_test(x, y):

    output, _ = forward_neuralnet(x)
    accuracy =  eval_accuracy(output, y)
    return accuracy

단층 퍼셉트론 순전파 역전파 함수 정의

In [141]:
'''

#Hyper Parameters

RND_MEAN = 0        #정규분포 난숫값의 평균
RND_STD = 0.0030    #정규분포 난숫값의 표준편차

LEARNING_RATE = 0.001 

def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x,weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose() 

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis =0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b


    '''

'\n\n#Hyper Parameters\n\nRND_MEAN = 0        #정규분포 난숫값의 평균\nRND_STD = 0.0030    #정규분포 난숫값의 표준편차\n\nLEARNING_RATE = 0.001 \n\ndef forward_neuralnet(x):\n    global weight, bias\n    output = np.matmul(x,weight) + bias\n    return output, x\n\ndef backprop_neuralnet(G_output, x):\n    global weight, bias\n    g_output_w = x.transpose() \n\n    G_w = np.matmul(g_output_w, G_output)\n    G_b = np.sum(G_output, axis =0)\n\n    weight -= LEARNING_RATE * G_w\n    bias -= LEARNING_RATE * G_b\n\n\n    '

실행하기

천체의 펄서 여부 판정 신경망 학습시키기

In [142]:
import csv

#pulsar_exec()

정확도 대신 신경망의 성능을 더 잘 보여줄 지표 , 정밀도(precision)와 재현율(recall)

In [143]:
#정밀도 : 신경망이 참으로 추정한 것 가운데 정답이 참인 비율 
#재현율 : 거꾸로, 정답이 참인 것들 가운데 신경망이 참으로 추정한 것의 비율 

#정밀도와 재현율 중 하나를 살피면 의미가 없다. 두 값을 함께 높여야한다. 
#-> F1 Score(=F1 value)를 사용 -> 정밀도와 재현율의 조화 평균(역수의 평균의 역수) -> F1 = 2*TP / ((2*TP) +FP + TN)


재정의 -> 메인실행함수, 데이터 적재함수, 정확도 계산함수

In [144]:
'''
def pulsar_exec(epoch_count=10, mb_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)   #데이터셋 Load
    init_model()            #모델 파라미터 초기화
    train_and_test(epoch_count, mb_size, report)        #학습 및 평가 과정 수행 

def load_pulsar_dataset(adjust_ratio):
    
    pulsars, stars = [], []
    with open('data/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)   #next함수는 csv.reader 다음에 자주쓰이는 함수임 다음 행을 읽는다 뭐 이런 의미라고 하는데 궁금하면 Googling 해보기
        rows = []

        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1         #입출력 벡터 크기
    
    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')
    
    
    #data = np.asarray(rows, dtype='float32')    #np.asarray(): 리스트 구조 -> 배열 구조 변환

def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data() 

    for epoch in range(epoch_count):
        losses =  []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, _ = run_train(train_x, train_y)
            losses.append(loss)

        if report > 0 and (epoch+1) % report ==0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {}: loss={:5.3f}, result={}'.format(epoch+1, np.mean(losses), acc_str))

    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    #final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final RESULT = {}'.format(acc_str))

'''
global est_yes, ans_yes, est_no, ans_no, tp, fp, fn, tn, accuracy
def eval_accuracy(output, y):
    global est_yes, ans_yes, est_no, ans_no, tp, fp, fn, tn, accuracy
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)

    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_no))
    tn = np.sum(np.logical_and(est_no, ans_yes))

    accuracy = safe_div(tp+fn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+tn)
    f1 = 2 * safe_div(recall*precision, recall+precision)

    return [accuracy, precision, recall, f1]

def safe_div(p, q):

    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q

    

In [145]:
pulsar_exec()

Epoch 1: loss=0.137, result=0.970,0.926,0.704,0.800
Epoch 2: loss=0.119, result=0.967,0.801,0.804,0.803
Epoch 3: loss=0.123, result=0.970,0.941,0.684,0.792
Epoch 4: loss=0.117, result=0.973,0.925,0.734,0.819
Epoch 5: loss=0.117, result=0.973,0.902,0.764,0.827
Epoch 6: loss=0.118, result=0.955,0.986,0.472,0.638
Epoch 7: loss=0.120, result=0.970,0.953,0.674,0.790
Epoch 8: loss=0.115, result=0.968,0.960,0.645,0.771
Epoch 9: loss=0.111, result=0.973,0.932,0.728,0.817
Epoch 10: loss=0.120, result=0.956,0.711,0.807,0.756

Final Test: final RESULT = 0.956,0.711,0.807,0.756


In [146]:
pulsar_exec(adjust_ratio=True)

Epoch 1: loss=0.403, result=0.920,0.930,0.909,0.920
Epoch 2: loss=0.385, result=0.786,0.721,0.943,0.817
Epoch 3: loss=0.383, result=0.924,0.968,0.880,0.922
Epoch 4: loss=0.374, result=0.927,0.961,0.891,0.925
Epoch 5: loss=0.382, result=0.909,0.980,0.838,0.903
Epoch 6: loss=0.378, result=0.777,0.702,0.974,0.816
Epoch 7: loss=0.380, result=0.921,0.940,0.902,0.920
Epoch 8: loss=0.365, result=0.924,0.938,0.911,0.924
Epoch 9: loss=0.374, result=0.922,0.946,0.897,0.921
Epoch 10: loss=0.365, result=0.923,0.977,0.868,0.919

Final Test: final RESULT = 0.923,0.977,0.868,0.919


In [147]:
est_yes, ans_yes, est_no, ans_no, tp, fp, fn, tn, accuracy

(array([[False],
        [ True],
        [False],
        ...,
        [False],
        [ True],
        [False]]),
 array([[False],
        [ True],
        [False],
        ...,
        [False],
        [ True],
        [False]]),
 array([[ True],
        [False],
        [ True],
        ...,
        [ True],
        [False],
        [ True]]),
 array([[ True],
        [False],
        [ True],
        ...,
        [ True],
        [False],
        [ True]]),
 2861,
 66,
 3146,
 435,
 0.9230178242163491)